# Lossless compression – Huffman coding

## Author @addobosz

In [33]:
from queue import PriorityQueue

In [34]:
def load_file(path):
    f = open(path, "r")
    return f.read()

In [35]:
norm_wiki_sample = load_file("norm_wiki_sample.txt")

In [36]:
def create_frequency_dict(text):
    freq_dict = {}
    for char in text:
        if char in freq_dict:
            freq_dict[char] += 1
        else:
            freq_dict[char] = 1
    return freq_dict

In [37]:
norm_wiki_sample_freq_dict = create_frequency_dict(norm_wiki_sample)

In [38]:
class Node():
    def __init__(self, key, val, left_node, right_node):
        self.key = key
        self.val = val
        self.left_node = left_node
        self.right_node = right_node

    def __gt__(self, other):
        return self.val > other.val
    
    def __lt__(self, other):
        return self.val < other.val
    
    def __repr__(self):
        return f"Node(key: {self.key}, freq: {self.val})"
    



In [71]:
class HuffmanTree:
    def __init__(self, freq_dict):
        self.freq_dict = freq_dict
        self.Q = PriorityQueue()
        self.root = self.create_tree()
        self.char_to_code_dict = {}
        self.code_to_char_dict = {}

    
    def create_tree(self):
        # initialize queue
        for key, value in self.freq_dict.items():
            self.Q.put(Node(key, value, None, None))
        
        # create tree    
        while self.Q.qsize() > 1:
            left = self.Q.get()
            right = self.Q.get()
            self.Q.put(Node(left.key + right.key, left.val + right.val, left, right))
        return self.Q.get()
    
    def create_code_dict(self):
        def traverse_tree(node, code):
            if node.left_node == None and node.right_node == None:
                # add code to dict only if node is a leaf
                self.char_to_code_dict[node.key] = code
            else:
                traverse_tree(node.left_node, code + '0')
                traverse_tree(node.right_node, code + '1')
        traverse_tree(self.root, '')

    def encode(self, text):
        if self.char_to_code_dict == {}:
            self.create_code_dict()

        encoded_text = ''
        for char in text:
            encoded_text += self.char_to_code_dict[char]

        return encoded_text


    def decode(self, code):
        # initialize dict mapping binary code to char
        if self.code_to_char_dict == {}:
            if self.char_to_code_dict == {}:
                self.create_code_dict()
            for key, value in self.char_to_code_dict.items():
                self.code_to_char_dict[value] = key
        
        decoded_text = ''
        
        curr_bit_sequence = ''
        for bit in code:
            curr_bit_sequence += bit
            if self.code_to_char_dict.get(curr_bit_sequence):
                decoded_text += self.code_to_char_dict[curr_bit_sequence]
                curr_bit_sequence = ''
            
        
        return decoded_text

In [74]:
tree = HuffmanTree(norm_wiki_sample_freq_dict)

Demonstration of encoding mechanism:

In [69]:
print(tree.encode("secret sequence"))

0011000010010101000101011100110001101101000110111000011101001000


Demonstration of decoding mechanism:

In [75]:
print(tree.decode("0011000010010101000101011100110001101101000110111000011101001000"))

secret sequence


Binary codes assigned to characters:

In [79]:
print(tree.char_to_code_dict)

{'e': '000', 'm': '00100', 'y': '001010', 'k': '0010110', '4': '001011100', 'x': '001011101', '5': '001011110', '3': '001011111', 's': '0011', 'w': '010000', 'b': '010001', 'c': '01001', 'r': '0101', 'o': '0110', 'n': '0111', 'i': '1000', 'd': '10010', '2': '10011000', '9': '10011001', 'v': '1001101', 'g': '100111', 't': '1010', 'p': '101100', 'f': '101101', 'l': '10111', 'a': '1100', 'h': '11010', '8': '110110000', 'j': '110110001', '0': '11011001', 'q': '1101101000', 'z': '1101101001', '6': '1101101010', '7': '1101101011', '1': '11011011', 'u': '110111', ' ': '111'}


Compress the text corpus, and compare it with the number of bits of a shortest possible fixed-length encoding:

In [81]:
print("Number of bits used to store the compressed text: ", len(tree.encode(norm_wiki_sample)))
print("Number of bits used to store the shortest possible fixed-length encoding (in this case, ASCII - one byte for each character): ", len(norm_wiki_sample) * 8)
print("Compression ratio: ", len(norm_wiki_sample) * 8 / len(tree.encode(norm_wiki_sample)))

Number of bits used to store the compressed text:  46489714
Number of bits used to store the shortest possible fixed-length encoding (in this case, ASCII - one byte for each character):  86311528
Compression ratio:  1.8565725743118144
